<h1> NEAT Implementation Practice </h1>
My Raw NEAT implementation, specifically for the mountaincar environment.

# Some Notes
Here are some things about which knobs you can turn in each cell:
1. Imports. Don't touch anything there unless you wanna try something out at your own risk.
2. Environment Declarations. Change the envName string if you want to test out other environments, but just note that you may need to change some other parameters in the evaluation cell as well if you do so.
3. Hyperparameters. Change as you please, but I would recommend sticking to the values from the NEAT paper and/or neatevolve.lua.
4. Some functions. I wouldn't touch this.
5. Genome Class. I wouldn't touch this, but please feel free to look at the mess I've made. I have recently moved the crossover and delta functions as static methods to this class.
8. Evaluation. As mentioned in 2), if you change the environment, then you made need to change the evaluateAction method of the genome or some of the target fitness requirements for episode termination.

In [14]:
import gym
import numpy as np # might not even need this ?
import random # pseudo-random|
import math
import time # for timing how long it runs for 
import PySimpleGUI as sg
import pickle 
import matplotlib.pyplot as plt

In [15]:
%matplotlib qt

In [16]:
# first define the environment and get the both the action and observation spaces 
envName = 'MountainCar-v0' # just need to change this if you wanna try it in environments
env = gym.make(envName)
plt.title(envName + ' Average Fitness across Generations')
# simple gui stuff
sg.theme = ('DarkAmber')
layout = [  [sg.Text('Trial: '), sg.Text(0, key='TRIAL-NUM', size= (6, 1))],
            [sg.Text('Simulation Info'), sg.Text(''),sg.Text('BestGenome Info')],
            [sg.Text('Generation: '), sg.Text(0, key='GENER-NUM', size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENER-NUM', size = (6,1))],
            [sg.Text('Genome: '), sg.Text(0, key='GENOM-NUM',size = (6,1)), sg.Text('Generation: '), sg.Text(0, key='BEST-GENOM-NUM', size = (6,1))],
            [sg.Text('Species: '), sg.Text(0, key='SPECIES-NUM',size = (6,1)), sg.Text('Species: '), sg.Text(0, key='BEST-SPECIES-NUM',size = (6,1))],
            [sg.Text('Fitness: '), sg.Text(0, key='FITNESS-NUM',size = (6,1)), sg.Text('Fitness: '), sg.Text(0, key='BEST-FITNESS-NUM',size = (6,1))]]



# # print out the observation and action spaces
# print(env.action_space.shape[0]) # number of valid actions we can take 
# print(env.observation_space.shape[0]) # number of "sensors" of the environment that we have (number of inputs)

# NOTE: Need to keep track of the valid values these can take 
# NOTE: Also need to have inputs + 1, with the 1 being the bias (stc?)
# NOTE: changes between Box() and Discrete() depending on environment
outputs = env.action_space.n
inputs = env.observation_space.shape[0] + 1


In [17]:
# hyperparameters, according to section 4.1 Parameter Settings (changed some parameters for myself)
deltaThreshold = 1.0

c1 = c2 = 2.0
c3 = 0.4
perturbVal = 0.025


numUntilStagnant = 15


genomeMutationChance = 0.25 # neatevolve.lua has this pretty low 
connectionUniformPerturb = 0.9
connectionRandomValue = 1 - connectionUniformPerturb
disableGeneChance = 0.75 
mutateNoCrossoverChance = 0.25
interspeciesMatingRate = 0.01 
smallPopulationNewNodeChance = 0.05
largePopulationNewNodeChance = 0.50 # using these right now 
smallPopulationNewConnectionChance = 0.03
largePopulationNewConnectionChance = 0.30 # using these right now 

In [18]:
# activation functions 
def sigmoid(x):
    return 1/(1+np.exp(-x))

def steepsigmoid(x):
    return 1/(1+np.exp(-4.9*x))

In [19]:
# genome class 
class Genome:
    # initialize class variables 
    globalInnovationNumber = 1
    def __init__(self, inputs=1, outputs=1):
        self.globalInnovation = 1
        self.nodeGenes = {}
        self.connectGenes = {}
        self.fitness = 0
        self.sharedFitness = 0
        self.species = 0
        self.inputCount = inputs
        self.outputCount = outputs
        self.mutationSuccess = False 
        
    def setGenes(self, nodeGenes, connectGenes):
        self.nodeGenes = nodeGenes.copy()
        self.connectGenes = connectGenes.copy()
        
    @staticmethod
    def copyGenome(genome):
        g = Genome(genome.inputCount, genome.outputCount)
        g.setGenes(genome.nodeGenes, genome.connectGenes)
        return g
    
    @staticmethod
    def resetGlobalInnovationNumber():
        Genome.globalInnovationNumber = 1
        # print('Genome global innovation number reset back to: {}'.format(Genome.globalInnovationNumber))
    
    @staticmethod
    def crossover(genome1, genome2):
        if (genome2.fitness > genome1.fitness):
            temp = genome1
            genome1 = genome2
            genome2 = temp

        g = Genome(genome1.inputCount, genome1.outputCount)
        g.initEmptyGenome()
        g1GenomeList = {}
        for connection in genome1.connectGenes:
            #print(genome1.connectGenes[connection])
            g1GenomeList[genome1.connectGenes[connection][0]]=[connection,genome1.connectGenes[connection][1],genome1.connectGenes[connection][2]]
        g2GenomeList = {}
        for connection in genome2.connectGenes:
            g2GenomeList[genome2.connectGenes[connection][0]]=[connection,genome2.connectGenes[connection][1],genome2.connectGenes[connection][2]] 

        # when crossing over, the genes in both genomes with the same innovation numbers are lined up
        # in composing the offspring, genes are randomly chosen from either parent at matching genes,
        # whereas all excess or disjoint genes are always included from the more fit parent 
        for i in range(max(len(g1GenomeList), len(g2GenomeList))):
            if (i+1 in g1GenomeList and i+1 in g2GenomeList):
                if (random.random() <= 50): # inherit from genome1
                    # print('inherited from parent 1')
                    g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
                else: # inherit from genome2 
                    # print('inherited from parent 2')
                    g.connectGenes[g2GenomeList[i+1][0]] = [i+1, g2GenomeList[i+1][1], g2GenomeList[i+1][2]]
                if (g1GenomeList[i+1][2] == False or g1GenomeList[i+1][2] == False):
                    if (random.random() <= disableGeneChance):
                        # print('inherited disabled gene')
                        g.connectGenes[g1GenomeList[i+1][0]][2] = False
                    else:
                        # print('re-enabled inherited gene')
                        g.connectGenes[g1GenomeList[i+1][0]][2] = True
            if (i+1 in g1GenomeList and i+1 not in g2GenomeList):
                g.connectGenes[g1GenomeList[i+1][0]] = [i+1, g1GenomeList[i+1][1], g1GenomeList[i+1][2]]
            # insert the node genes 

        for c in g.connectGenes:
            if c[0] in g1.nodeGenes:
                g.nodeGenes[c[0]] = g1.nodeGenes[c[0]]
            elif c[0] in g2.nodeGenes:
                g.nodeGenes[c[0]] = g2.nodeGenes[c[0]]

            if c[1] in g1.nodeGenes:
                g.nodeGenes[c[1]] = g1.nodeGenes[c[1]]
            elif c[0] in g2.nodeGenes:
                g.nodeGenes[c[1]] = g2.nodeGenes[c[1]]

        g.clearNodeValues()
        return g
    
    @staticmethod
    def delta(genome1, genome2, c1=0.1, c2=0.1, c3=0.1):
        if (not genome1.connectGenes or not genome2.connectGenes):
            return 0
        connectionMismatch = []
        for connection1 in genome1.connectGenes:
            if (connection1 not in genome2.connectGenes):
                connectionMismatch.append(connection1)
        for connection2 in genome2.connectGenes:
            if (connection2 not in genome1.connectGenes):
                connectionMismatch.append(connection2)
        diffsum = 0
        num = 0
        for connection1 in genome1.connectGenes:
            if (connection1 not in connectionMismatch):
                diffsum += abs(genome1.connectGenes[connection1][1] -
                               genome2.connectGenes[connection1][1])
                num += 1
        averageWeightDifference = diffsum / num
        #print(averageWeightDifference)
        N = 0
        if len(genome1.connectGenes) >= len(genome2.connectGenes):
            N = len(genome1.connectGenes)
        else:
            N = len(genome2.connectGenes)
        print(((len(connectionMismatch)*(c1 + c2))/N) + (c3*averageWeightDifference))
        return ((len(connectionMismatch) *
                 (c1 + c2)) / N) + (c3 * averageWeightDifference)
    
    def clearGenome(self):
        self.globalInnovation = 1
        self.nodeGenes.clear()
        self.connectGenes.clear()
        
    def clearNodeValues(self):
        for node in self.nodeGenes:
            self.nodeGenes[node][1] = 0.0
            
    def initEmptyGenome(self):
        self.clearGenome()
        for i in range(self.inputCount):
            self.insertNode('input')
        for i in range(self.outputCount):
            self.insertNode('output')
            
    def resetFitness(self):
        self.fitness = 0
        self.sharedFitness = 0
        
    def initRandomGenome(self):
        self.initEmptyGenome()
        
        # brute force connect each new input node to each output node 
        for node in self.nodeGenes:
            if self.nodeGenes[node][0] == 'input':
                for otherNode in self.nodeGenes:
                    if self.nodeGenes[otherNode][0] == 'output':
                        self.insertConnection(outNode=node, inNode=otherNode, weight=random.uniform(-1, 1))
             
    def insertNode(self, nodeType='input', value = 0.0):
        self.nodeGenes[len(self.nodeGenes)+1]=[nodeType, value]
        
    def insertConnection(self, outNode, inNode, weight = 0.5, isExpressed = True):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes): # check if the nodes even exist 
            if ((outNode, inNode) not in self.connectGenes and (inNode, outNode) not in self.connectGenes and self.nodeGenes[inNode][0] != 'input' and outNode != inNode and self.nodeGenes[outNode][0] != 'output'): # check that the connection doesnt already exist
                # also dont allow recurrent connections (only feedforward)
                self.connectGenes[(outNode, inNode)] = [Genome.globalInnovationNumber, weight, isExpressed]
                Genome.globalInnovationNumber += 1
                self.mutationSuccess = True
            else:
                return
                #print('Connection already exists. Did not insert')
        else:
            return
            #print('Could not find node(s). Did not insert.')
            
    def checkConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                return self.connectGenes[(outNode, inNode)][2]
            else:
                print('Could not find the connection')
        else:
            print('Could not find node(s). Did not disable.')
            
    def disableConnection(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.connectGenes[(outNode, inNode)][2] = False
            else:
                print('Could not find connection to disable. Did not disable.')
        else: 
            print('Could not find node(s). Did not disable.')
    
    def mutateAddConnection(self, outNode, inNode):
        self.insertConnection(outNode, inNode, random.uniform(-1, 1), True)
            
    def mutateAddRandomConnection(self):
        timesAttemptedToConnected = 0
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            self.mutateAddConnection(random.choice(list(self.nodeGenes)),random.choice(list(self.nodeGenes)))
            timesAttemptedToConnected +=1
            if (timesAttemptedToConnected > 100):
                print('Attempted to connect too many times. probably already full')
                return
        self.mutationSuccess = False
        
    def mutateAddNode(self, outNode, inNode):
        if (outNode in self.nodeGenes) and (inNode in self.nodeGenes):
            if (outNode, inNode) in self.connectGenes:
                self.insertNode('hidden', '0.0')
                # disable old connection'
                self.disableConnection(outNode, inNode)
                self.insertConnection(outNode, len(self.nodeGenes), 1.0, True)
                # get original weight 
                originalWeight = self.connectGenes[(outNode, inNode)][1]
                self.insertConnection(len(self.nodeGenes), inNode, originalWeight, True)
                self.mutationSuccess = True
                print('added hidden node')
            else:
                print('Error! No connection found.')
        else:
            print('Could not find node(s). Did not mutate.')
    
    def mutateAddRandomNode(self):
        if (not self.connectGenes): # need to debug this 
            print('connectGenes is empty. can\'t mutateAddRandomNode')
            return
        self.mutationSuccess = False
        while (not self.mutationSuccess):
            randomTuple = random.choice(list(self.connectGenes))
            self.mutateAddNode(randomTuple[0],randomTuple[1])
        self.mutationSuccess = False
    
    # this is so fucking slow
    def evaluateAction(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output.index(max(output))
        
    def evaluateActionBox(self, observation):
        # TODO: IMPLEMENT THIS
        if not self.connectGenes:
            return env.action_space.sample()
        else:
            self.clearNodeValues()
            self.nodeGenes[1][1] = 1.0
            for i in range(self.inputCount-1):
                # print('Loading value: {} into node input: {}'.format(observation[i], i+2))
                self.nodeGenes[i+2][1] = observation[i]
            
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'hidden': # for every hidden node
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True: # if inNode == node
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = steepsigmoid(np.dot(x, w))
                    # print('hidden node #{}\'s value: {}'.format(node, self.nodeGenes[node][1]))
            softmaxsum = 0
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    x = []
                    w = []
                    for connection in self.connectGenes:
                        if connection[1] == node and self.connectGenes[connection][2] == True:
                            x.append(self.nodeGenes[connection[0]][1])
                            w.append(self.connectGenes[connection][1])
                    self.nodeGenes[node][1] = np.exp(np.dot(x, w))
                    softmaxsum += np.exp(np.dot(x, w))
                    # print('output node #{}\'s unweighted value: {}'.format(node, self.nodeGenes[node][1]))
            output = []
            for node in self.nodeGenes:
                if self.nodeGenes[node][0] == 'output':
                    output.append(self.nodeGenes[node][1] / softmaxsum)
            # print(output)
            # print('Genome takes action {}'.format(output.index(max(output))))
            return output
    
    def evaluateSharedFitness(self, speciesCount):
        self.sharedFitness = self.fitness / speciesCount
    
    def printNodeGenes(self):
        print(self.nodeGenes)
        
    def printNodeGeneCount(self):
        print(len(self.nodeGenes))
        
    def printConnectGenes(self):
        print(self.connectGenes)
    
    def printConnectGeneCount(self):
        print(len(self.connectGenes))
    
    def printActiveConnectGenes(self):
        for connection in self.connectGenes:
            if (self.connectGenes[connection][2]):
                print(connection, self.connectGenes[connection][1])
        
    def printGlobalInnovation(self):
        print(self.globalInnovation)

> connection weights mutate as in any NE system, with each connection either perturbed or not at each generation 

In [20]:
max_generations = 50
population_limit = 150
championThresholdCount = 5

inputs = env.observation_space.shape[0] + 1
outputs = env.action_space.n

bestGenomeDictionary = {}

targetPosition = 0.5
maxTestTrials = 100
targetAverageFitness = -110.0

window = sg.Window(envName, layout, finalize=True)
start_time = time.time()

globalEvaluationCounter = 0
solvedCounter = 0

# NOTE: input/output changes depending on environment !

for q in range(maxTestTrials):
    hl, = plt.plot([],[])
    window['TRIAL-NUM'].update(q+1)
    species = {}
    pop = {}
    solved = False
    trialLoopCounter = 0
    bestGenome = None
    highestFitness = float("-inf")
    for generation in range(max_generations):
        generationFitness = 0
        print('Beginning Generation #{}'.format(generation+1))
        window['GENER-NUM'].update(generation+1)
        if (generation == 0 and not pop): # start out with a uniform population of networks with zero hidden networks
            for genome in range(population_limit):
                g = Genome(inputs, outputs)
                Genome.resetGlobalInnovationNumber()
                g.initRandomGenome()
                pop[genome+1] = g

        # in each generation, genomes are sequentially placed into species 

        fitness = []

        for genome in pop:
            print('Beginning evaluation on genome #{}'.format(genome))
            window['GENOM-NUM'].update(genome)
            if (not species):
                print('creating new species:{}'.format(len(species)+1))
                species[len(species)+1] = {'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter' : 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                pop[genome].species = len(species)
                print(len(species))
            else:
                for og in species:
                    if (Genome.delta(pop[genome], species[og]['list'][0], c1, c2, c3) < deltaThreshold): # just compare with the first species
                        print('inserting into species#{}'.format(og))
                        species[og]['list'].append(pop[genome])
                        pop[genome].species = og
                        break
                    else:
                        if (og == len(species)):
                            print('creating new species:{}'.format(len(species)+1))
                            species[len(species)+1] ={'summedSharedFitness': 0.0, 'maxFitness':0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': 0, 'isStagnant':False,'list':[pop[genome]], 'bestGenome': pop[genome]}
                            pop[genome].species = len(species)
                            print(len(species))
                            break

            window['SPECIES-NUM'].update(pop[genome].species)
            observation = env.reset()
            pop[genome].resetFitness()
            # evalute the genome's performance # SDHJKLSDHLSJKDHLSDHJKLHDSKLDHJ
            t = 0
            trialLoopCounter += 1
            while(True):
                t+=1
                env.render()
                action = pop[genome].evaluateAction(observation)
                observation, reward, done, info = env.step(action)
                pop[genome].fitness += reward
                window['FITNESS-NUM'].update(pop[genome].fitness)
                window.finalize()
                if done:
                    print("Episode finished after {} timesteps".format(t+1))
                    print("Genome fitness: {}".format(pop[genome].fitness))
                    fitness.append(pop[genome].fitness)
                    generationFitness+=pop[genome].fitness
                    if pop[genome].fitness > species[pop[genome].species]['maxFitness']:
                        species[pop[genome].species]['maxFitness'] = pop[genome].fitness
                        species[pop[genome].species]['stagnantCounter'] = 0
                        species[pop[genome].species]['isStagnant'] = False
                    if pop[genome].fitness > highestFitness:
                        bestGenome = pop[genome]
                        highestFitness = pop[genome].fitness
                        # update the window values 
                        window['BEST-GENER-NUM'].update(generation+1)
                        window['BEST-GENOM-NUM'].update(genome)
                        window['BEST-SPECIES-NUM'].update(pop[genome].species)
                        window['BEST-FITNESS-NUM'].update(pop[genome].fitness)
                        window.finalize()
                    if observation[0] >= targetPosition: # changes depending on environment !
                        print('Found an optimal genome. testing it...')
                        testBenchTotalFitness = 0
                        for i in range(maxTestTrials):
                            m = 0
                            pop[genome].resetFitness()
                            observation = env.reset()
                            while(True):
                                m+=1
                                env.render()
                                action = pop[genome].evaluateAction(observation)
                                observation, reward, done, info = env.step(action)
                                pop[genome].fitness += reward
                                window['FITNESS-NUM'].update(pop[genome].fitness)
                                window.finalize()
                                if done:
                                    testBenchTotalFitness+=pop[genome].fitness
                                    print("Genome fitness: {}".format(pop[genome].fitness))
                                    break
                        print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/maxTestTrials))
                        if (testBenchTotalFitness/maxTestTrials >= targetAverageFitness):
                            print('found solution genome')
                            solved = True
                            solvedCounter +=1
                            bestGenomeDictionary[len(bestGenomeDictionary)+1] = pop[genome]
                    break
            if (solved):
                break
        if (solved):
            break
            
        # update the plot of the average fitness across generations 
        hl.set_xdata(np.append(hl.get_xdata(), generation))
        hl.set_ydata(np.append(hl.get_ydata(), generationFitness/population_limit))
        plt.plot(hl.get_xdata(), hl.get_ydata())
        plt.draw()
        print('Number of species at generation {}: {}'.format(generation+1, len(species)))
        # print('highest achieved score in this generation {}'.format(max(fitness)))

        # print(species[1])
        adjustedFitness = []
        totalAdjustedFitness = 0.0
        for genome in pop:
            pop[genome].evaluateSharedFitness(len(species[pop[genome].species]))
            adjustedFitness.append(pop[genome].sharedFitness)
            species[pop[genome].species]['summedSharedFitness'] += pop[genome].sharedFitness
            if pop[genome].sharedFitness > species[pop[genome].species]['maxAdjustedFitness']:
                species[pop[genome].species]['maxAdjustedFitness'] = pop[genome].sharedFitness
                species[pop[genome].species]['bestGenome'] = pop[genome]

        for s in species:
            totalAdjustedFitness += species[s]['summedSharedFitness']
            print('Maximum adjusted fitness in species {}: {}'.format(s, species[s]['maxAdjustedFitness']))
            print('Summed adjusted fitness in species {}: {}'.format(s, species[s]['summedSharedFitness']))

        # print('Total Adjusted Fitness across all species: {}'.format(totalAdjustedFitness))

        newPop = {}

        # the champion of each species with more than 5 networks will be copied into the next generation unchanged
        i = 0
        for s in species:
            if len(species[s]['list']) > championThresholdCount:
                newPop[i+1] = Genome.copyGenome(species[s]['bestGenome'])
                i += 1

        print('New Pop count after copying champions only: {}'.format(len(newPop)))
        print('i\'s value after inserting champions: {}'.format(i))


        # getting the proportion of offspring they are allowed
        totalBreedCount = 0
        for s in species:
            if species[s]['isStagnant']:
                print('species #{} is stagnant'.format(s))
                species[s]['breedAmountAllowed'] = 0
            else:
                species[s]['breedAmountAllowed'] = np.floor((species[s]['summedSharedFitness']/totalAdjustedFitness)*(population_limit-i)).astype(int)
                totalBreedCount += species[s]['breedAmountAllowed']


        leftovers = population_limit - i - totalBreedCount
        print('leftovers:{}'.format(leftovers))
        
        # get rid of the "lowest performing members" of the population
        amountToCull = np.floor(.75*len(pop)).astype(int)
        print('Amount to cull: {}'.format(amountToCull))
        # print(sorted(pop.items(), key=lambda x: x[1].fitness))
        for b in range(amountToCull):
            tempTupleList = sorted(pop.items(), key=lambda x:x[1].fitness, reverse = True)
            tempGenomeKey = tempTupleList.pop()[0]
            tempGenome = pop.pop(tempGenomeKey)
            species[tempGenome.species]['list'].remove(tempGenome)
            print('Removed Genome # {} from species # {}, with a fitness of {}, from the population'.format(tempGenomeKey, tempGenome.species, tempGenome.fitness))


        # now for each species breed !
        for s in species:
            if (len(species[s]['list']) >= 1):
                print('species {} is allowed to have {} offspring'.format(s, species[s]['breedAmountAllowed']))
                for j in range(species[s]['breedAmountAllowed']):
                    print('making offspring #{}'.format(i+1))
                    if (random.random() <= mutateNoCrossoverChance or len(species[s]['list'])==1): # either the genome is a crossover or just the same genome 
                        print('Chose no crossover')
                        g = Genome.copyGenome(random.choice(species[s]['list']))
                    else:
                        print('Chose crossover')
                        g1 = random.choice(species[s]['list'])
                        g2 = random.choice(species[s]['list'])
                        g = Genome.crossover(g1, g2)

                    # add a chance to mutate the connections
                    if (g.connectGenes):
                        if (random.random() <= genomeMutationChance):
                            print('mutation genome connections !')
                            for connection in g.connectGenes:
                                if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                    g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                                else:
                                    g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 


                    # chance to add new connections/nodes
                    if (random.random() <= largePopulationNewNodeChance):
                        print('trying to add new node')
                        g.mutateAddRandomNode()
                    if (random.random() <= largePopulationNewConnectionChance):
                        print('trying to add new connection')
                        g.mutateAddRandomConnection()

                    newPop[i+1] = g
                    i+=1
            else:
                print('could not breed species{}, list is empty !'.format(s))
        # -----------------------------------------------------------------------------------------------------        

        if (leftovers > 0): 
            print('now for the leftovers:')
        for k in range(leftovers): # breeding the leftover genomes that were not calculated through floor
            if (random.random() <= interspeciesMatingRate): # very small chance
                print('interspecies')
                speciesKey1 = random.choice(list(species))
                speciesKey2 = random.choice(list(species))
                if (len(species[speciesKey1]['list']) < 1 or len(species[speciesKey2]['list']) < 1):
                    print('either species does not have enough to breed. skipping.')
                    k -= 1
                    continue
                g1 = random.choice(species[speciesKey1]['list'])
                g2 = random.choice(species[speciesKey2]['list'])
                g = Genome.crossover(g1, g2)
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        print('mutation genome connections !')
                        for connection in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 


                # chance to add new connections/nodes
                if (random.random() <= largePopulationNewNodeChance):
                    print('trying to add new node')
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    print('trying to add new connection')
                    g.mutateAddRandomConnection()


                newPop[i+1] = g
                i+=1
            else: # else just find a random species and mate from there 
                print('random species mating')
                speciesKey = random.choice(list(species))
                if (len(species[speciesKey]['list']) < 1):
                    print('species does not have enough to breed. skipping.')
                    k -= 1
                    continue
                if (random.random() <= mutateNoCrossoverChance or len(species[speciesKey]['list']) < 2): # either the genome is a crossover or just the same genome 
                    print('Chose no crossover')
                    g = Genome.copyGenome(random.choice(species[speciesKey]['list']))
                else:
                    print('Chose crossover')
                    g1 = random.choice(species[speciesKey]['list'])
                    g2 = random.choice(species[speciesKey]['list'])
                    g = Genome.crossover(g1, g2)

                # add a chance to mutate the connections
                if (g.connectGenes):
                    if (random.random() <= genomeMutationChance):
                        print('mutation genome connections !')
                        for connection in g.connectGenes:
                            if (random.random() <= connectionUniformPerturb): # uniformly perturb the connection
                                g.connectGenes[connection][1] += random.uniform(-perturbVal, perturbVal)
                            else:
                                g.connectGenes[connection][1] = random.uniform(-1.0, 1.0) # set a new value for 


                # chance to add new connections/nodes
                if (random.random() <= largePopulationNewNodeChance):
                    print('trying to add new node')
                    g.mutateAddRandomNode()
                if (random.random() <= largePopulationNewConnectionChance):
                    print('trying to add new connection')
                    g.mutateAddRandomConnection()


                newPop[i+1] = g
                i+=1


        print('Final value of i: {}'.format(i))

        # ------------------------------------------------------------------------------------------

        # each existing species is represented by a random genome inside the species from the previous generation 
        if (generation >= 0):
            for s in species:
                species[s] = {'summedSharedFitness': 0.0, 'maxFitness': 0.0, 'maxAdjustedFitness': 0.0, 'stagnantCounter': species[s]['stagnantCounter']+1, 'isStagnant': False, 'list' : [random.choice(species[s]['list'])]}
                species[s]['bestGenome'] = species[s]['list'][0]
                if species[s]['stagnantCounter'] >= numUntilStagnant:  # if the maximum fitness of a species did not improve in 15 generations, the networks in the stagnant species were not allowed to reproduce 
                    species[s]['isStagnant'] = True




        pop.clear()
        pop = newPop.copy()


    print('Total number of evaluations for this trial: {}'.format(trialLoopCounter))
    globalEvaluationCounter += trialLoopCounter 

print("--- %s seconds ---" % (time.time() - start_time))
print('{} solved out of {} trials'.format(solvedCounter, maxTestTrials))    
print('Total Number of evaluations overall {}'.format(globalEvaluationCounter))
print('Average evaluation count per trial: {}'.format(globalEvaluationCounter / maxTestTrials))
window.close()
env.close()
# saving contents of population into pickle when it is finished 
with open('data/pop.dictionary', 'wb') as config_dictionary_file:
    pickle.dump(pop, config_dictionary_file)
    
with open('data/bestGenome_'+envName+'.dictionary', 'wb') as dictionary_file:
    pickle.dump(bestGenomeDictionary, dictionary_file)

Beginning Generation #1
Beginning evaluation on genome #1
creating new species:1
1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #2
0.2243839412375649
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #3
0.1433186338267129
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #4
0.24881349872520744
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #5
0.16629276197539686
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #6
0.2964565385816928
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #7
0.17470989106679097
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on ge

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #61
0.21668383676047234
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #62
0.29103236832001766
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #63
0.19670361447489848
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #64
0.22269560282134881
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #65
0.31188155185627536
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #66
0.2085477615720393
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #67
0.2704989596829637
inserting into species#1
Episode finished after

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #120
0.23925080723831088
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #121
0.28120949972049275
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #122
0.1744662991913099
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #123
0.24411979229937333
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #124
0.3078992263726333
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #125
0.2175658217857179
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #126
0.3126655153829657
inserting into species#1
Episode finished 

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #2
0.2671060301020469
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #3
0.23102128152448742
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #4
0.255027347574382
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #5
0.2252633123821367
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #6
0.2686587296758729
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #7
0.3270452835919231
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #8
0.35178916466402804
inserting into species#1
Episode finished after 201 timest

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #58
0.21849186166770443
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #59
0.15628960027030128
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #60
0.926659293696
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #61
0.2671060301020469
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #62
1.2219460731565404
1.591969214820801
1.3420658252271613
0.6369105470621615
inserting into species#4
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #63
0.2188342518028708
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #64
0.189215012493777

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #113
0.35302197673588287
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #114
0.36972162113933366
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #115
0.24649321324506293
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #116
1.0200015797777124
1.2664180445660558
0.46115159801569905
inserting into species#3
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #117
1.2184918616677045
1.6048149427718421
1.3611048921939821
1.3061924408933796
1.1352802629424128
0.30920934968908753
inserting into species#6
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #118
0.9959314569486002
inserting into species#1
Episode finished after

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #2
0.5482754827425551
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #3
1.2026587611137667
1.5478803037458595
0.9518898308446484
inserting into species#3
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #4
0.9709826480504422
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #5
1.981626643133124
1.583872428111774
0.9348496075028322
inserting into species#3
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #6
0.9472152301897736
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #7
0.9716389167135363
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #8
0.93

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #56
0.9716389167135363
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #57
0.9560242742038443
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #58
1.859596640900095
0.8090853492193152
inserting into species#2
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #59
0.9988395746041583
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #60
0.9988395746041583
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #61
1.195799396623617
1.5314755671787084
1.5510658062633402
0.5635021802376368
inserting into species#4
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #

Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #110
0.8303267647938372
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #111
1.7702247368494366
1.0
1.298982258787016
1.0329121792194282
0.2730150103620279
inserting into species#5
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #112
0.9472152301897736
inserting into species#1
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #113
1.3030578571051132
1.3333333333333333
0.9420983625904913
inserting into species#3
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #114
1.300283054959501
0.0
inserting into species#2
Episode finished after 201 timesteps
Genome fitness: -200.0
Beginning evaluation on genome #115
1.9823459489706488
0.7379641807921037
inserting into species#2
Episode finished after 201 timesteps
Gen

IndexError: Cannot choose from an empty sequence

In [ ]:
# load the networks that solved the problem !
with open('data/bestGenome_cartpole.dictionary', 'rb') as dictionary_file:
    bestGenomeDictionary = pickle.load(dictionary_file)

In [ ]:
# test bench 

env = gym.make(envName)

testBenchTotalFitness = 0

for i in range(100):
    t = 0
    bestGenome.resetFitness()
    observation = env.reset()
    while(True):
        t+=1
        env.render()
        action = bestGenome.evaluateAction(observation)
        observation, reward, done, info = env.step(action)
        bestGenome.fitness += reward
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            testBenchTotalFitness+=bestGenome.fitness
            print("Genome fitness: {}".format(bestGenome.fitness))
            break

print('average fitness across 100 trials:{}'.format(testBenchTotalFitness/100))
env.close()